In [4]:
!pip install enelvo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 64.3 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234910 sha256=fa8272ef0ddff05a214d20b2f375dd44880f6a46cb739f740f2de89c2e62d176
  Stored in directory: /root/.cache/pip/wheels/02/3d/88/51a592b9ad17e7899126563698b4e3961983ebe85747228ba6
Successfully built emoji


In [5]:
from google.colab import drive
from enelvo.normaliser import Normaliser
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import nltk

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Pré-processamento

## 0) Base de Dados

In [7]:
norm = Normaliser(tokenizer='readable')
chats_clients = pd.read_csv('/content/drive/Shareddrives/grupo3moshi/chats_clients.csv')
chats_clients.head()

,No,Intencao,Pergunta,Resposta
0,1,Como depositar,Boa dia.tudo bem?eu gostaria de saber sobre aq...,"Bom dia! Sim, o sr pode utilizar o cartão de d..."
1,2,Como fazer remessa,Como enviar dinheiro do Japão?,"Para se inscrever no serviço de remessa, por f..."
2,3,Tempo de remessa,Quanto tempo levará para o beneficiário recebe...,"Via de regra, as remessas serão pagas via PIX ..."
3,4,"Pedido de envio via metodo ""ByPhone""",Boa tarde\nAcabei de fazer a transferência de ...,iremos processar a sua solicitacao.\nMuito obr...
4,5,"Pedido de envio via metodo ""ByPhone""",Poderia fazer a remessa de 22yenes para o BBB ...,iremos processar a sua solicitacao.\nMuito obr...


In [8]:
chats_clients.fillna('erro ao processar a pergunta', inplace=True)

In [9]:
import random
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Exemplo de troca de ordem das palavras
def shuffle_words(sentence):
    words = sentence.split()
    random.shuffle(words)
    return ' '.join(words)

# Exemplo de remoção de stopwords
def remove_stopwords(sentence):
    stop_words = set(stopwords.words('portuguese'))
    word_tokens = word_tokenize(sentence)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    return ' '.join(filtered_sentence)

# Aplicando as técnicas
chats_clients['Pergunta_Shuffled'] = chats_clients['Pergunta'].apply(lambda x: shuffle_words(x))
chats_clients['Resposta_Shuffled'] = chats_clients['Resposta'].apply(lambda x: shuffle_words(x))


# Mostrar as primeiras linhas após as técnicas de data augmentation
chats_clients[['Pergunta', 'Pergunta_Shuffled', 'Resposta', 'Resposta_Shuffled']].head()


,Pergunta,Pergunta_Shuffled,Resposta,Resposta_Shuffled
0,Boa dia.tudo bem?eu gostaria de saber sobre aq...,qualquer mart verde caixa dia.tudo hora?obriga...,"Bom dia! Sim, o sr pode utilizar o cartão de d...",Bom máquinas o utilizar horas o cartão pode Ba...
1,Como enviar dinheiro do Japão?,dinheiro Como enviar do Japão?,"Para se inscrever no serviço de remessa, por f...",dados já app poderá https://brastelremit.jp/po...
2,Quanto tempo levará para o beneficiário recebe...,dinheiro? o o beneficiário levará receber Quan...,"Via de regra, as remessas serão pagas via PIX ...","as Via dados pagas serão PIX de regra, remessa..."
3,Boa tarde\nAcabei de fazer a transferência de ...,"transferência de de Total 22+23, Boa Acabei ta...",iremos processar a sua solicitacao.\nMuito obr...,Muito obrigada otima a e processar solicitacao...
4,Poderia fazer a remessa de 22yenes para o BBB ...,o de AAAA Obrigado remessa econômica E MMMM VV...,iremos processar a sua solicitacao.\nMuito obr...,otima e processar sua Muito tarde. solicitacao...


In [10]:
import random
from nltk.corpus import wordnet

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace("_", " "))
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)

def synonym_replacement(sentence, n=2):
    words = sentence.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if len(get_synonyms(word)) > 0]))
    random.shuffle(random_word_list)

    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(synonyms)
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break

    return ' '.join(new_words)

# Exemplo de aplicação
chats_clients['Pergunta_Augmented'] = chats_clients['Pergunta'].apply(lambda x: synonym_replacement(x))
chats_clients['Resposta_Augmented'] = chats_clients['Resposta'].apply(lambda x: synonym_replacement(x))

# Mostrar as primeiras linhas após a substituição de sinônimos
chats_clients[['Pergunta', 'Pergunta_Augmented', 'Resposta', 'Resposta_Augmented']].head()


,Pergunta,Pergunta_Augmented,Resposta,Resposta_Augmented
0,Boa dia.tudo bem?eu gostaria de saber sobre aq...,Boa dia.tudo bem?eu gostaria de cavalry sword ...,"Bom dia! Sim, o sr pode utilizar o cartão de d...","Bom dia! Sim, o strontium pode utilizar o cart..."
1,Como enviar dinheiro do Japão?,Como enviar dinheiro coiffure Japão?,"Para se inscrever no serviço de remessa, por f...","Para se inscrever no serviço de remessa, por f..."
2,Quanto tempo levará para o beneficiário recebe...,Quanto pacing levará parity o beneficiário rec...,"Via de regra, as remessas serão pagas via PIX ...","Via First State regra, as remessas serão pagas..."
3,Boa tarde\nAcabei de fazer a transferência de ...,Boa tarde Acabei Diamond State fazer A transfe...,iremos processar a sua solicitacao.\nMuito obr...,iremos processar group A sua solicitacao. Muit...
4,Poderia fazer a remessa de 22yenes para o BBB ...,Poderia fazer A remessa de 22yenes para o BBB ...,iremos processar a sua solicitacao.\nMuito obr...,iremos processar A sua solicitacao. Muito obri...


In [11]:
# Criar um novo DataFrame para armazenar os dados aumentados
df_augmented = pd.DataFrame()

# Adicionar as linhas originais
df_augmented = chats_clients.copy()

# Adicionar as linhas de perguntas e respostas embaralhadas
df_augmented_extra1 = pd.DataFrame({
    'No': chats_clients['No'],
    'Intencao': chats_clients['Intencao'],
    'Pergunta': chats_clients['Pergunta_Shuffled'],
    'Resposta': chats_clients['Resposta_Shuffled']
})

df_augmented_extra2 = pd.DataFrame({
    'No': chats_clients['No'],
    'Intencao': chats_clients['Intencao'],
    'Pergunta': chats_clients['Pergunta_Augmented'],
    'Resposta': chats_clients['Resposta_Augmented']
})
# Concatenar o DataFrame original com o DataFrame de data augmentation
df_augmented = pd.concat([df_augmented, df_augmented_extra1, df_augmented_extra2], ignore_index=True)

# Remover as colunas auxiliares de shuffle
df_augmented = df_augmented[['No', 'Intencao', 'Pergunta', 'Resposta']]

# Exibir as primeiras linhas do DataFrame final
df_augmented.shape


(1515, 4)

In [12]:
# Especificar o caminho onde o arquivo CSV será salvo
output_file_path = '/content/drive/Shareddrives/grupo3moshi/augmented_data.csv'

# Salvar o DataFrame aumentado como um arquivo CSV
df_augmented.to_csv(output_file_path, index=False)
